### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Apache Spark + Delta Lake

<b>Delta Lake é um storage layer de código aberto que traz confiabilidade aos Data Lakes. O Delta Lake fornece transações ACID, scalable metadata handlig. O Delta Lake é executado no Data Lake existente e é compatível com as APIs do Apache Spark.</b>

## Importando Bibliotecas e Criando o Spark Session

In [1]:
# Import da Bibliotecas
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from delta import *

In [2]:
# Criação do Spark Session

builder = pyspark.sql.SparkSession.builder.appName("AppInLabDelta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Carregando Dataset para o DataFrame

In [3]:
dataset = "/home/datascience/DeltaInLab/dataset/characters_sw.csv"

df = spark.read\
        .format("csv")\
        .option("delimiter", ";")\
        .option("InferSchema", "true")\
        .option("header", "true")\
        .load(dataset)

In [4]:
df.show()

+---+----------------+------+-----+----------+-----------+---------+----------+------+---------+--------------+
| ID|            name|height| mass|hair_color| skin_color|eye_color|birth_year|gender|homeworld|       species|
+---+----------------+------+-----+----------+-----------+---------+----------+------+---------+--------------+
| 10|  Luke Skywalker|   172| 77.0|     blond|       fair|     blue|     19BBY|  male| Tatooine|         Human|
| 20|           C-3PO|   167| 75.0|        NA|       gold|   yellow|    112BBY|    NA| Tatooine|         Droid|
| 30|           R2-D2|    96| 32.0|        NA|white, blue|      red|     33BBY|    NA|    Naboo|         Droid|
| 40|     Darth Vader|   202|136.0|      none|      white|   yellow|   41.9BBY|  male| Tatooine|         Human|
| 50|     Leia Organa|   150| 49.0|     brown|      light|    brown|     19BBY|female| Alderaan|         Human|
| 60|  Obi-Wan Kenobi|   182| 77.0|     white|       fair|blue-gray|     57BBY|  male|  Stewjon|        

## Criando Estrutura Delta Table

In [5]:
df.select("ID", "name", "height").write.format("delta").save("/home/datascience/DeltaInLab/deltalab-table")

In [6]:
read_delta = spark.read.format("delta").load("/home/datascience/DeltaInLab/deltalab-table")
read_delta.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- height: integer (nullable = true)



In [8]:
deltaTable = DeltaTable.forPath(spark, "/home/datascience/DeltaInLab/deltalab-table")
deltaTable.toDF().show()

+---+----------------+------+
| ID|            name|height|
+---+----------------+------+
| 10|  Luke Skywalker|   172|
| 20|           C-3PO|   167|
| 30|           R2-D2|    96|
| 40|     Darth Vader|   202|
| 50|     Leia Organa|   150|
| 60|  Obi-Wan Kenobi|   182|
| 70|Anakin Skywalker|   188|
| 80|       Chewbacca|   228|
| 90|        Han Solo|   180|
|100|            Yoda|    66|
|110|       Palpatine|   170|
|120|       Boba Fett|   183|
+---+----------------+------+



# Delta Lake Features

## Schema Management - Schema Enforcement

In [9]:
deltaTable.toDF().show(2)

+---+--------------+------+
| ID|          name|height|
+---+--------------+------+
| 10|Luke Skywalker|   172|
| 20|         C-3PO|   167|
+---+--------------+------+
only showing top 2 rows



In [10]:
read_delta.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- height: integer (nullable = true)



In [11]:
nschema = read_delta.schema

new_schema_deltaTable = spark.createDataFrame([(130, "Jabba Desilijic Tiure", 175)], nschema).withColumn("ID",expr("cast(ID as String)"))

new_schema_deltaTable.write.format("delta").mode("append").save("/home/datascience/DeltaInLab/deltalab-table")

AnalysisException: Failed to merge fields 'ID' and 'ID'. Failed to merge incompatible data types IntegerType and StringType

In [12]:
new_schema_deltaTable.printSchema

<bound method DataFrame.printSchema of DataFrame[ID: string, name: string, height: int]>

In [13]:
spark.read.format("delta").load("/home/datascience/DeltaInLab/deltalab-table").printSchema()

root
 |-- ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- height: integer (nullable = true)



## Schema Management - Schema Evolution

In [14]:
nschema = read_delta.schema

se_deltaTable = spark.createDataFrame([(130, "Jabba Desilijic Tiure", 175)], nschema).withColumn("species", lit("Hutt"))

se_deltaTable.write.format("delta").option("mergeSchema", "true").mode("append").save("/home/datascience/DeltaInLab/deltalab-table")

In [15]:
spark.read.format("delta").load("/home/datascience/DeltaInLab/deltalab-table").show()


+---+--------------------+------+-------+
| ID|                name|height|species|
+---+--------------------+------+-------+
|130|Jabba Desilijic T...|   175|   Hutt|
| 10|      Luke Skywalker|   172|   null|
| 20|               C-3PO|   167|   null|
| 30|               R2-D2|    96|   null|
| 40|         Darth Vader|   202|   null|
| 50|         Leia Organa|   150|   null|
| 60|      Obi-Wan Kenobi|   182|   null|
| 70|    Anakin Skywalker|   188|   null|
| 80|           Chewbacca|   228|   null|
| 90|            Han Solo|   180|   null|
|100|                Yoda|    66|   null|
|110|           Palpatine|   170|   null|
|120|           Boba Fett|   183|   null|
+---+--------------------+------+-------+



In [16]:
spark.read.format("delta").load("/home/datascience/DeltaInLab/deltalab-table").where("ID = 130").show()

+---+--------------------+------+-------+
| ID|                name|height|species|
+---+--------------------+------+-------+
|130|Jabba Desilijic T...|   175|   Hutt|
+---+--------------------+------+-------+



# Delta Lake Features
## Update - Inserts - Merge - Deletes

In [17]:
deltaTable = DeltaTable.forPath(spark, "/home/datascience/DeltaInLab/deltalab-table")

#Update

deltaTable.update(
        condition = expr("ID = 10"),
        set = {"species" : lit("Human")}            
)

deltaTable.toDF().show()

+---+--------------------+------+-------+
| ID|                name|height|species|
+---+--------------------+------+-------+
| 10|      Luke Skywalker|   172|  Human|
| 20|               C-3PO|   167|   null|
| 30|               R2-D2|    96|   null|
| 40|         Darth Vader|   202|   null|
| 50|         Leia Organa|   150|   null|
| 60|      Obi-Wan Kenobi|   182|   null|
| 70|    Anakin Skywalker|   188|   null|
| 80|           Chewbacca|   228|   null|
| 90|            Han Solo|   180|   null|
|100|                Yoda|    66|   null|
|110|           Palpatine|   170|   null|
|120|           Boba Fett|   183|   null|
|130|Jabba Desilijic T...|   175|   Hutt|
+---+--------------------+------+-------+



In [19]:
# Delete

#deltaTable.delete(condition = expr("ID = 110"))

#deltaTable.toDF().where("ID = 110").show()
deltaTable.toDF().show()

+---+--------------------+------+-------+
| ID|                name|height|species|
+---+--------------------+------+-------+
| 10|      Luke Skywalker|   172|  Human|
| 20|               C-3PO|   167|   null|
| 30|               R2-D2|    96|   null|
| 40|         Darth Vader|   202|   null|
| 50|         Leia Organa|   150|   null|
| 60|      Obi-Wan Kenobi|   182|   null|
| 70|    Anakin Skywalker|   188|   null|
| 80|           Chewbacca|   228|   null|
| 90|            Han Solo|   180|   null|
|100|                Yoda|    66|   null|
|120|           Boba Fett|   183|   null|
|130|Jabba Desilijic T...|   175|   Hutt|
+---+--------------------+------+-------+



In [20]:
deltaTable.toDF().where("ID in (20, 30, 40)").show()

+---+-----------+------+-------+
| ID|       name|height|species|
+---+-----------+------+-------+
| 20|      C-3PO|   167|   null|
| 30|      R2-D2|    96|   null|
| 40|Darth Vader|   202|   null|
+---+-----------+------+-------+



In [21]:
# Merge

schema = deltaTable.toDF().schema

dfupdate = spark.createDataFrame([
    (20, "C-3PO", 167, "Droid"),
    (30, "R2-D2", 96, "Droid"),
    (40, "Darth Vader", 202, "Human"),
    (110, "Palpatine", 110, "Human")
],schema)

deltaTable.alias("primary").merge(dfupdate.alias("updates"), "primary.ID = updates.ID")\
                           .whenMatchedUpdate(set = {"name" : col("updates.name"), "height" : col("updates.height"), "species" : col("updates.species")})\
                           .whenNotMatchedInsertAll()\
                           .execute()

deltaTable.toDF().where("ID in (20, 30, 40, 110)").show()

+---+-----------+------+-------+
| ID|       name|height|species|
+---+-----------+------+-------+
| 20|      C-3PO|   167|  Droid|
| 30|      R2-D2|    96|  Droid|
| 40|Darth Vader|   202|  Human|
|110|  Palpatine|   110|  Human|
+---+-----------+------+-------+



# Delta Lake Features

## Time Travel

In [22]:
deltaTable.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      4|2023-02-17 17:40:50|  null|    null|    MERGE|{predicate -> (pr...|null|    null|     null|          3|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.2....|
|      3|2023-02-17 17:37:28|  null|    null|   DELETE|{predicate -> ["(...|null|    null|     null|          2|  Serializable|        false|{numRemovedFiles ...|        null|Apache-Spark/3.2....|
|      2|2023-0

In [25]:
#spark.read.format("delta").option("versionAsOf", "4").load("/home/datascience/DeltaInLab/deltalab-table").show()

spark.read.format("delta").option("timestampAsOf", "2023-02-17 17:37:28").load("/home/datascience/DeltaInLab/deltalab-table").show()


+---+--------------------+------+-------+
| ID|                name|height|species|
+---+--------------------+------+-------+
| 10|      Luke Skywalker|   172|  Human|
| 20|               C-3PO|   167|   null|
| 30|               R2-D2|    96|   null|
| 40|         Darth Vader|   202|   null|
| 50|         Leia Organa|   150|   null|
| 60|      Obi-Wan Kenobi|   182|   null|
| 70|    Anakin Skywalker|   188|   null|
| 80|           Chewbacca|   228|   null|
| 90|            Han Solo|   180|   null|
|100|                Yoda|    66|   null|
|120|           Boba Fett|   183|   null|
|130|Jabba Desilijic T...|   175|   Hutt|
+---+--------------------+------+-------+



In [24]:
spark.read.format("delta").option("versionAsOf", "3").load("/home/datascience/DeltaInLab/deltalab-table").show()

+---+--------------------+------+-------+
| ID|                name|height|species|
+---+--------------------+------+-------+
| 10|      Luke Skywalker|   172|  Human|
| 20|               C-3PO|   167|   null|
| 30|               R2-D2|    96|   null|
| 40|         Darth Vader|   202|   null|
| 50|         Leia Organa|   150|   null|
| 60|      Obi-Wan Kenobi|   182|   null|
| 70|    Anakin Skywalker|   188|   null|
| 80|           Chewbacca|   228|   null|
| 90|            Han Solo|   180|   null|
|100|                Yoda|    66|   null|
|120|           Boba Fett|   183|   null|
|130|Jabba Desilijic T...|   175|   Hutt|
+---+--------------------+------+-------+



# Obrigada!!

Delta - https://delta.io/
<br>
Delta Getting Started - https://delta.io/learn/getting-started
<br>
OCI Data Science - https://www.oracle.com/br/artificial-intelligence/data-science/